# Dark Sky API (Weather)

Answer the following questions using the [Dark Sky API](https://darksky.net/dev/). I've added three cells for each question but you're free to use more or less! Hold `Shift` and hit `Enter` to run a cell, and use the `+` on the top left to add a new cell to a notebook.

If you spend some time reading the documentation your life will probably be a little bit easier!

## 0) Import any libraries you might need

- *Tip: We're going to be downloading things from the internet, so we probably need `requests`.*

In [1]:
import requests

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
secretapikey = os.getenv("darksky_apikey")
base_url = "https://api.darksky.net/forecast/"

## 1) Make a request from the Dark Sky API for where you were born (or lived, or want to visit!).

- *Tip: The URL we used in class was for a place near San Francisco. What was the format of the endpoint that made this happen?*
- *Tip: Save the URL as a separate variable, and be sure to not have `[` and `]` inside.*
- *Tip: How is north vs. south and east vs. west latitude/longitude represented? Is it the normal North/South/East/West?*
- *Tip: You know it's JSON, but Python doesn't! Make sure you aren't trying to deal with plain text.* 
- *Tip: Once you've imported the JSON into a variable, check the timezone's name to make sure it seems like it got the right part of the world!*

In [3]:
latlon_phila = ("39.96"+","+"-75.18")
weather_phila = requests.get(f'{base_url}{secretapikey}/{latlon_phila}').json()

In [4]:
print(weather_phila.keys())

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'minutely', 'hourly', 'daily', 'flags', 'offset'])


In [5]:
#weather_phila['daily']

In [6]:
now_phila = weather_phila['currently']
now_phila

{'time': 1560229021,
 'summary': 'Overcast',
 'icon': 'cloudy',
 'nearestStormDistance': 22,
 'nearestStormBearing': 136,
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 72.96,
 'apparentTemperature': 74.5,
 'dewPoint': 71.94,
 'humidity': 0.97,
 'pressure': 1010.34,
 'windSpeed': 3.48,
 'windGust': 4.32,
 'windBearing': 222,
 'cloudCover': 0.99,
 'uvIndex': 0,
 'visibility': 8.19,
 'ozone': 314.61}

## 2) What's the current wind speed? How much warmer does it feel than it actually is?

- *Tip: You can do this by browsing through the dictionaries, but it might be easier to read the documentation*
- *Tip: For the second half: it **is** one temperature, and it **feels** a different temperature. Calculate the difference.*

In [7]:
print(f"Windspeed is {now_phila['windSpeed']:.1f} kpm (so {now_phila['windSpeed'] * 0.539957:.1f} knots, i guess?).")

Windspeed is 3.5 kpm (so 1.9 knots, i guess?).


In [8]:
now_temp = int(now_phila['temperature'])
now_tempapp = int(now_phila['apparentTemperature'])
now_tempdif = now_temp - now_tempapp
if now_tempdif > 0:
    feels = (f"{abs(now_tempdif)} warmer.")
elif now_tempdif < 0:
    feels = (f"{abs(now_tempdif)} cooler.")
else:
    feels = "just like it."
print(f"The temperature is currently {now_temp} degrees but it feels {feels}.")

The temperature is currently 72 degrees but it feels 2 cooler..


## 3) Today's forecast is the FIRST daily forecast. For the place you decided on above, how much of the moon is currently visible?

- *Tip: This does **not** use the `currently` key.*
- *Tip: It might be helpful to save the forecast into its own variable*

In [9]:
daily_phila = weather_phila['daily']['data']
daily_phila[0]

{'time': 1560225600,
 'summary': 'Mostly cloudy until afternoon.',
 'icon': 'partly-cloudy-day',
 'sunriseTime': 1560245580,
 'sunsetTime': 1560299438,
 'moonPhase': 0.3,
 'precipIntensity': 0.0071,
 'precipIntensityMax': 0.0792,
 'precipIntensityMaxTime': 1560236400,
 'precipProbability': 0.71,
 'precipType': 'rain',
 'temperatureHigh': 79.75,
 'temperatureHighTime': 1560286800,
 'temperatureLow': 58.84,
 'temperatureLowTime': 1560330000,
 'apparentTemperatureHigh': 79.75,
 'apparentTemperatureHighTime': 1560286800,
 'apparentTemperatureLow': 58.84,
 'apparentTemperatureLowTime': 1560330000,
 'dewPoint': 55.91,
 'humidity': 0.63,
 'pressure': 1014.01,
 'windSpeed': 9.64,
 'windGust': 28.37,
 'windGustTime': 1560247200,
 'windBearing': 328,
 'cloudCover': 0.55,
 'uvIndex': 8,
 'uvIndexTime': 1560276000,
 'visibility': 9.43,
 'ozone': 325.3,
 'temperatureMin': 62.35,
 'temperatureMinTime': 1560254400,
 'temperatureMax': 79.75,
 'temperatureMaxTime': 1560286800,
 'apparentTemperatureMin'

In [10]:
print(f"In its current phase, {int(daily_phila[0]['moonPhase'] * 100)}% of the moon should be visible.")

In its current phase, 30% of the moon should be visible.


## 4) What's the difference between the high and low temperatures for today?

In [11]:
tempdif = daily_phila[0]['temperatureHigh'] - daily_phila[0]['temperatureLow']

In [12]:
print(f"There's about a {abs(tempdif):.0f} difference in today's high & low temps.")

There's about a 21 difference in today's high & low temps.


## 5) Go through the daily forecasts, printing out the next week's worth of predictions.

I'd like to know the **high temperature** for each day, and whether it's **hot, warm, or cold** (based on what temperatures you think are hot, warm or cold).

- *Tip: Is it returning F or C? You can change them by adding `?units=us` or `?units=si` onto the end of the URL.*
- *Tip: You'll need to use an `if` statement to say whether it is hot, warm or cold.*

In [13]:
from datetime import datetime
import calendar
degree_sign = u'\N{DEGREE SIGN}'

In [21]:
for day in daily_phila:
    day['dt'] = datetime.fromtimestamp(day['time'])
    day['dt_clean'] = (day['dt']).strftime('%c').replace(' 00:00:00',',')
    day['dt_clean_long'] = (day['dt']).strftime('%A, %B %d')
#     print(day['dt'])
#     print(type(day['dt']))
#     print(day['dt_clean'])
#     print(type(day['dt_clean']))

In [15]:
print("This week's forecast is:")
for day in daily_phila:
    if day['temperatureMax'] >= 95:
        temp_desc = 'hot'
    elif float(day['temperatureMax']) >= 81:
        temp_desc = 'warm'
    elif float(day['temperatureMax']) >= 70:
        temp_desc = 'pleasant' 
    elif float(day['temperatureMax']) >= 45:
        temp_desc = 'cool'
    elif float(day['temperatureMax']) < 45:
        temp_desc = 'cold' 
    print(f"\t{day['dt_clean']}: {temp_desc}, high of {int(day['temperatureMax'])}{degree_sign}")

This week's forecast is:
	Tue Jun 11, 2019: pleasant, high of 79°
	Wed Jun 12, 2019: pleasant, high of 75°
	Thu Jun 13, 2019: pleasant, high of 71°
	Fri Jun 14, 2019: pleasant, high of 73°
	Sat Jun 15, 2019: warm, high of 83°
	Sun Jun 16, 2019: warm, high of 89°
	Mon Jun 17, 2019: warm, high of 92°
	Tue Jun 18, 2019: warm, high of 87°


## 6) What will be the hottest day in the next week? What is the high temperature on that day?

It's okay if your result looks like `1453363200`. If you want to be extra-fancy, though, format it as a **real date** - the weird number is called a **UNIX timestamp**, which might help in googling.

- *Tip: It's okay to skip this if it seems too difficult!*

In [16]:
week_highs = [float(day['temperatureMax']) for day in daily_phila]
week_high = max(week_highs)

In [17]:
print(f"The warmest day of the week will be {day['dt_clean_long']} with a high of {int(week_high)}{degree_sign}.")

The warmest day of the week will be Tuesday, June 18 with a high of 92°.


## 7) What's the weather looking like for the next 24+ hours in Miami, Florida?

I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 50% say "{temperature} and cloudy" instead of just the temperature. 


- *Tip: Be sure to remember how latitude and longitude are done with positive and negative numbers*
- *Tip: How do you represent "50%" for cloud cover?*

In [20]:
latlon_miami = ("25.76"+","+"-80.19")
weather_miami = requests.get(f'{base_url}{secretapikey}/{latlon_miami}').json()
hourly_miami = weather_miami['hourly']['data']
hourly_miami[0]

{'time': 1560229200,
 'summary': 'Humid',
 'icon': 'clear-night',
 'precipIntensity': 0.0002,
 'precipProbability': 0.01,
 'precipType': 'rain',
 'temperature': 79.28,
 'apparentTemperature': 83.06,
 'dewPoint': 73.81,
 'humidity': 0.83,
 'pressure': 1017.3,
 'windSpeed': 2.87,
 'windGust': 4.28,
 'windBearing': 234,
 'cloudCover': 0,
 'uvIndex': 0,
 'visibility': 10,
 'ozone': 289.25}

In [ ]:
print(f"Here is the weather forecast in miami for the next 24 hours:")
for hour in hourly_miami:
    hour['dt'] = datetime.fromtimestamp(day['time'])
    hour['dt_clean'] = (day['dt']).strftime('%c').replace(' 00:00:00',',')
    hour['dt_clean_long'] = (day['dt']).strftime('%A, %B %d')#.replace(' 00:00:00',',')
for hour in hourly_miami:
    if hour['temperatureMax'] >= 95:
        temp_desc = 'hot'
    elif float(hour['temperatureMax']) >= 81:
        temp_desc = 'warm'
    elif float(hour['temperatureMax']) >= 70:
        temp_desc = 'pleasant' 
    elif float(hour['temperatureMax']) >= 45:
        temp_desc = 'cool'
    elif float(hour['temperatureMax']) < 45:
        temp_desc = 'cold' 
    print(f"\t{day['dt_clean']}: {temp_desc}, high of {int(day['temperatureMax'])}{degree_sign}")

## 8) For the next 24-ish hours in Miami, what percent of the time is the temperature above 85 degrees?

If they give you 48 hours instead of 24, that's okay.

- *Tip: You might want to read up on [looping patterns](http://jonathansoma.com/lede/foundations-2017/classes/data%20structures/looping-patterns/)*

## 9) What was the temperature in Central Park on Christmas Day, 2012? How about 2007? 2005? How far back does the API allow you to go?

- *Tip: You'll need to use UNIX time, which is the number of seconds since January 1, 1970. Google can help you convert a normal date!*
- *Tip: You'll want to use Forecast.io's "time machine" API, [documentation here](https://darksky.net/dev/docs/time-machine)*